In [1]:
# ライブラリのインストール
%pip install transformers fugashi ipadic sentencepiece

from transformers import BertJapaneseTokenizer, BertModel
import torch
import numpy as np


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\WorkSpace\Practice\env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # モデル出力の最初の要素がトークン埋め込みを含む
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        for batch_idx in range(0, len(sentences), batch_size):
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="max_length", max_length=512,
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings)

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [3]:

# テキストファイルを読み込み、各文をリストに格納
def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = f.read().splitlines()
    return sentences

# モデルの初期化
model1 = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")
model2 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese")
model3 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-v3")
model4 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-char-v3")
model5 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-v2")
model6 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-char-v2")



# テキストファイルから文を読み込む
file_path = "E:\実験\実験結果/18.文章25\文章25.txt"  # 読み込むファイルのパス
sentences = read_sentences_from_file(file_path)

# 文をエンコード
embeddings1 = model1.encode(sentences)
embeddings2 = model2.encode(sentences)
embeddings3 = model3.encode(sentences)
embeddings4 = model4.encode(sentences)
embeddings5 = model5.encode(sentences)
embeddings6 = model6.encode(sentences)


c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

In [14]:
for i in range(1, len(sentences)):
    print(sentences[i])

無論、今日のように学問が個々の諸学問に分かれる以前は、学問をすること自体が哲学をするのと同じ意味だった。
このように、歴史的起源あるいは発生的観点から、哲学が全ての学問の根本であるのは事実である。
しかし、哲学が学問の根本と呼べる本源的な理由は別にある。
それは哲学が基本的に個々の学問の土台となっている根本的な概念や前提そのものに対して疑問を提起し、それらをテーマ的に扱うという事実に起因する。
いくつかの例を挙げてみることにしよう。
基本的に「科学」はこの世界で起こる、ある事象を“説明”することを目指す。
 しかし、もし誰かがこの世界で起きる、ある具体的な事象を科学的に説明しようとするのではなく、「科学的説明」自体が何かについて疑問を提起し、科学的理論がどのように「客観性」を確保できるかについて明らかにしようとするならば、それは科学ではなく、「科学哲学」である。
自然科学に関する一例を挙げよう。
 私たちは高校の化学の授業で、物質の根本単位は「原子」と学んできた。
 しかし、原子のような小さい物質は私たちの肉眼で“直接的に観察”することが不可能である。
ただ、電子顕微鏡を通じてその存在をただ間接的に類推できるだけだ。
 それでは、「原子」というのは本当に存在するのだろうか。
それとも、それはただ、ある事態を効果的に説明するために理論的に仮定されたフィクションに過ぎないのだろうか。
もし、誰かがこのような質問に対して悩み始めたとしたら、その人は科学で前提にしている理論的対象に対する形而上学的、存在論的問題について悩んでいると見ることができる。
それなら、原子というものが本当に存在すると仮定してみよう。
 ところが、先に述べたように、原子を直接的に肉眼で観察することが不可能ならば、原子というものが本当に存在するということを、私たちはどうやって知ることができるだろうか。
 電子顕微鏡によって間接的に原子の存在を類推するだけで、私たちは原子が存在するということが本当に”分かる”と言えるだろうか。 
このような疑問は、原子よりも小さい電子や微粒子の存在に対しても提起することができる。
 もし誰かがこのような問題について悩み始めたら、その人は科学で想定している理論的対象に対して認識論的に悩んでいると言える。
これに似た疑問を社会科学分野にも適用させることができる。
 一社会を構

In [ ]:
len(sentences)

In [4]:

## コサイン類似度を計算して出力
#for i in range(len(sentences) - 1):
#    print(f'sentence {i+1} vs sentence {i+2}: {cos_sim(embeddings[i].numpy(), embeddings[i+1].numpy())}')

# コサイン類似度を計算して出力
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings1[i].numpy(), embeddings1[i+1].numpy())}')


0.717995822429657
0.7272423505783081
0.6650372743606567
0.7151206731796265
0.3032328188419342
0.22000528872013092
0.6526998281478882
0.5056691765785217
0.2958485186100006
0.4588942527770996
0.4985571801662445
0.3285762071609497
0.25539544224739075
0.45571252703666687
0.26467159390449524
0.7296499013900757
0.8287378549575806
0.5560721755027771
0.3734455704689026
0.4850529134273529
0.36868712306022644
0.666556715965271
0.6671707630157471
0.6232078075408936
0.5612947940826416
0.6764847040176392
0.5508853793144226
0.5671570301055908
0.7556020617485046
0.637042760848999
0.6241620182991028


In [5]:
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings2[i].numpy(), embeddings2[i+1].numpy())}')

0.8971399664878845
0.9105226993560791
0.9269354939460754
0.9155671000480652
0.7874377965927124
0.7896202802658081
0.9304020404815674
0.8186363577842712
0.7984268665313721
0.9097556471824646
0.9113258123397827
0.8532882332801819
0.9050958752632141
0.9219511151313782
0.9018036127090454
0.9199346303939819
0.9541264772415161
0.8849017024040222
0.8844093680381775
0.875779390335083
0.801954448223114
0.8979100584983826
0.8867316246032715
0.8845264911651611
0.9015142321586609
0.9426397681236267
0.9354618191719055
0.9455174803733826
0.8760387301445007
0.8752005100250244
0.8787786960601807


In [6]:
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings3[i].numpy(), embeddings3[i+1].numpy())}')

0.9398021697998047
0.9503487944602966
0.9559738636016846
0.9332430958747864
0.7878365516662598
0.801395833492279
0.9187604188919067
0.8410629630088806
0.8662056922912598
0.9153521656990051
0.9322462677955627
0.8648903965950012
0.9116213917732239
0.9113568067550659
0.8991959691047668
0.9246852993965149
0.9575814008712769
0.9194684624671936
0.9082335233688354
0.9032925963401794
0.8349844813346863
0.9064933061599731
0.9156196713447571
0.9196091890335083
0.9315834045410156
0.9516934752464294
0.9445369839668274
0.957012414932251
0.9247029423713684
0.9280030131340027
0.9122307896614075


In [7]:
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings4[i].numpy(), embeddings4[i+1].numpy())}')

0.9257162809371948
0.9472770690917969
0.9692095518112183
0.947005569934845
0.839110791683197
0.8563038110733032
0.9258238673210144
0.8276437520980835
0.886405348777771
0.9493907690048218
0.9553306698799133
0.889599084854126
0.9253969192504883
0.9459165930747986
0.9343342185020447
0.9371994137763977
0.9754616022109985
0.9450754523277283
0.946420431137085
0.9450122714042664
0.8869732618331909
0.9455122947692871
0.9394050240516663
0.9390310645103455
0.9390324950218201
0.9668002128601074
0.9594862461090088
0.964759111404419
0.9241932034492493
0.9367245435714722
0.9049847722053528


In [8]:
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings5[i].numpy(), embeddings5[i+1].numpy())}')

0.9155191779136658
0.9219727516174316
0.9439801573753357
0.9203893542289734
0.7713755369186401
0.7771182060241699
0.8940684199333191
0.8465381860733032
0.8006554245948792
0.8911910653114319
0.9189766049385071
0.8420080542564392
0.8895239233970642
0.9142545461654663
0.8782067894935608
0.9200230240821838
0.9482157826423645
0.9023969769477844
0.8768181204795837
0.9076787233352661
0.850246250629425
0.8428640961647034
0.8673362135887146
0.9164651036262512
0.9266574382781982
0.9432199001312256
0.9237576127052307
0.9431870579719543
0.9037277698516846
0.9041284918785095
0.8592539429664612


In [9]:
for i in range(len(sentences) - 1):
    print(f'{cos_sim(embeddings6[i].numpy(), embeddings6[i+1].numpy())}')

0.9150168895721436
0.9271034598350525
0.9600526094436646
0.9244741797447205
0.7977563142776489
0.8157901167869568
0.9142224788665771
0.8416613936424255
0.8589282631874084
0.9125257730484009
0.9247131943702698
0.8445374369621277
0.8923085331916809
0.9235914349555969
0.8955807685852051
0.9149519801139832
0.9587867856025696
0.9158143997192383
0.8982867002487183
0.916883111000061
0.8736933469772339
0.928999662399292
0.921089231967926
0.9220705628395081
0.9244064092636108
0.9546757340431213
0.9443618059158325
0.9463220834732056
0.922421395778656
0.911771297454834
0.8723496794700623
